In [1]:
import json
from seq2seq_translation.config.transformer_config import TransformerConfig

with open('/Users/adam.amster/Downloads/encoder_decoder_config.json') as f:
    config = json.load(f)
config = TransformerConfig.model_validate(config)

In [2]:
import torch
from seq2seq_translation.run import _fix_model_state_dict
from seq2seq_translation.models.transformer.encoder_decoder import EncoderDecoderTransformer


def construct_model():
	model = EncoderDecoderTransformer(
        n_attention_heads=config.n_head,
        n_layers=config.num_layers,
        vocab_size=30000,
        d_model=config.d_model,
        block_size=config.fixed_length,
        feedforward_hidden_dim=config.feedforward_hidden_dim,
        norm_first=config.norm_first,
        mlp_activation=config.activation,
        positional_encoding_type=config.positional_encoding_type,
        sos_token_id=1,
        eos_token_id=2,
        pad_token_id=3
	)

	model.load_state_dict(
		_fix_model_state_dict(torch.load('/Users/adam.amster/Downloads/ckpt.pt', map_location='cpu')["model"])
	)

	model.eval()
	return model
model = construct_model()

In [3]:
model.num_params

59499520

In [20]:
from collections import Counter
cnt = Counter()
for n, p in model.named_parameters():
    if p.requires_grad:
        cnt[p.shape] += p.numel()

# top 15 largest shapes
for shp, num in cnt.most_common(15):
    print(f"{str(shp):>18} : {num/1e6:6.2f} M")
print(f"TOTAL                 : {sum(cnt.values())/1e6:6.2f} M")

torch.Size([30000, 512]) :  30.72 M
torch.Size([2048, 512]) :  12.58 M
torch.Size([512, 2048]) :  12.58 M
torch.Size([1536, 512]) :   9.44 M
torch.Size([512, 512]) :   6.29 M
torch.Size([1024, 512]) :   3.15 M
torch.Size([128, 512]) :   0.13 M
 torch.Size([512]) :   0.05 M
torch.Size([2048]) :   0.02 M
torch.Size([1536]) :   0.02 M
torch.Size([1024]) :   0.01 M
TOTAL                 :  74.99 M


In [35]:
6 * 65*10**6*50000*100000/1e18

1.95

In [43]:
6*158*10**6*6.1*10**9

5.7828e+18